In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

# 1. Télécharger les données
print("Importing data...")
data = yf.download("^GSPC", start="1985-01-01", end="2023-01-01")

# --- CORRECTION DU BUG ---
# Si le téléchargement échoue (vide), on arrête
if data.empty:
    raise ValueError("Erreur: Aucune donnée téléchargée. Vérifie ta connexion internet.")

# Gestion du MultiIndex (Si yfinance met le ticker en entête de colonne)
# Ex: Colonne ('Adj Close', '^GSPC') au lieu de 'Adj Close'
if isinstance(data.columns, pd.MultiIndex):
    print("Structure MultiIndex détectée : Nettoyage des colonnes...")
    data.columns = data.columns.get_level_values(0)

# Vérification du nom de la colonne (Parfois 'Adj Close' n'existe pas, on prend 'Close')
col_name = 'Adj Close'
if col_name not in data.columns:
    print(f"Attention: '{col_name}' absent. Utilisation de 'Close' à la place.")
    col_name = 'Close'
# -------------------------

# 2. Calcul des Log-Rendements
data['Log_Return'] = np.log(data[col_name] / data[col_name].shift(1))

# 3. Nettoyage (supprimer les NaN créés par le shift)
data = data.dropna()

# 4. Sauvegarde pour le C++
data['Log_Return'].to_csv("input_returns.txt", index=False, header=False)
# Sauvegarde des dates pour les graphiques futurs
data.index.to_series().to_csv("input_dates.csv", index=False, header=False)

print("Succès ! Fichiers 'input_returns.txt' et 'input_dates.csv' générés.")
print(f"Aperçu des rendements :\n{data['Log_Return'].head()}")

Importing data...


/var/folders/sg/d13nr7xd2s799kmlk23319kw0000gn/T/ipykernel_64581/249569955.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download("^GSPC", start="1985-01-01", end="2023-01-01")
[*********************100%***********************]  1 of 1 completed

Structure MultiIndex détectée : Nettoyage des colonnes...
Attention: 'Adj Close' absent. Utilisation de 'Close' à la place.
Succès ! Fichiers 'input_returns.txt' et 'input_dates.csv' générés.
Aperçu des rendements :
Date
1985-01-03   -0.004849
1985-01-04   -0.005423
1985-01-07    0.003416
1985-01-08   -0.001523
1985-01-09    0.007230
Name: Log_Return, dtype: float64
